<a href="https://colab.research.google.com/github/aquapathos/openCVLesson/blob/master/IP/%E3%83%A2%E3%83%BC%E3%83%89%E6%B3%95%E3%81%A8%E9%83%A8%E5%88%86%E7%94%BB%E5%83%8F%E5%88%86%E5%89%B2%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import urllib
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from matplotlib import pylab
from google.colab import widgets
import warnings
import matplotlib
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)

# １．モード法による画像の２階調化

In [0]:
# pict01 とそのヒストグラムの表示

url1 = "https://monosnap.com/image/VcgFAtIa6RFIgg6VEqLuqIH8RL2IaI" # pict01 画像URL

img_path, header = urllib.request.urlretrieve(url1)
img1 = cv2.imread(img_path,0) #  画像の読み込み
cv2_imshow(img1) #
print("図１　処理対象画像 pict01")
cv2.imwrite("pict01.png",img1)
  
hist1 = cv2.calcHist([img1],[0],None,[256],[0,256])
plt.hist(img1.ravel(),256,[0,256]); plt.show()
print("図２　pict01 の濃度ヒストグラム")

## 課題1　

pict01 のヒストグラムをもとに、モード法によりしきい値を決め、２階調化しなさい。

In [0]:
しきい値 = 191 #@param {type:"slider", min:0, max:255, step:1}
ret,bwimg1 = cv2.threshold(img1,しきい値,255,cv2.THRESH_BINARY)
cv2_imshow(bwimg1) #
print("          図３     ２階調化画像　　しきい値{}\n".format(しきい値))
cv2.imwrite("pict01BIN.png",bwimg1);


#２.　ヒストグラムが双峰ではない画像の２階調化

In [0]:
# pict02 とそのヒストグラムの表示

url2 = "https://monosnap.com/image/NLCngPegjHp6RdYqSUyPiTLTGW1pPa" # pict02 画像URL

img_path, header = urllib.request.urlretrieve(url2)
img2 = cv2.imread(img_path,0) #  画像の読み込み
cv2_imshow(img2) #
print("図４　処理対象画像 pict02")
cv2.imwrite("pict02.png",img2)
  
hist2 = cv2.calcHist([img2],[0],None,[256],[0,256])
plt.hist(img2.ravel(),256,[0,256]); plt.show()
print("図５　pict02 の濃度ヒストグラム")



　pict02のヒストグラムは双峰ではなく、３つ以上の山が見られる。

　このように山が複数あっても、例えば、右の二つの山が、明るさは異なるが、どちらも背景色であり、文字がその暗い方よりもさらに暗いのであれば、右から2つ目の谷の濃度をしきい値に選んでやることで、文字は黒、背景は白に2階調化できる可能性はある。

## 課題２-1 
pict02 のヒストグラムには４つの山がある。谷の部分３か所の濃度を読み取り、次のプログラムで、それらの値をしきい値にセットして、２階調化した画像を作りなさい。

そして、得られた結果が、どのような結果となったか、それはなぜなのかをそれぞれ考察し、説明しなさい。


In [0]:
#@title Number fields
しきい値１ = 100 #@param {type:"slider", min:0, max:255, step:1}
しきい値２ = 153 #@param {type:"slider", min:0, max:255, step:1}
しきい値３ = 226 #@param {type:"slider", min:0, max:255, step:1}
ths=[しきい値１,しきい値2,しきい値3]
  
for i in range(3):
    ret,bwimg2 = cv2.threshold(img2,ths[i],255,cv2.THRESH_BINARY)
    cv2_imshow(bwimg2) #
    print("          図{} pict02の２階調化画像　　しきい値{}\n".format(i+6,ths[i]))

## 課題３　
pict02 から文字の掠れや黒つぶれのないきれいな２値画像を得るためには、授業で習ったどの手法が使えるか、その手法はどういう手法で、なぜその方法ならば pict02 がうまく処理できるのかを述べよ。


----

----


# 発展　

以下は課題ではないが、実行してみよ。

## 移動平均法による２階調化

In [0]:
#@title Number fields
offset = 13 #@param {type:"slider", min:-30, max:20, step:1}
ksize = 31 #@param {type:"slider", min:3, max:40, step:2}
th3 = cv2.adaptiveThreshold(img2,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,ksize,offset)
cv2_imshow(th3) #
print("          図９     移動平均法の結果　")

# 自動　部分画像分割法

[GrabCut](http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_core/py_image_arithmetics/py_image_arithmetics.html#id5) を応用して自動的に左右分割を行う


In [0]:
img = cv2.imread(img_path,1)
cv2_imshow(img)
print('図10　入力画像（再掲）')

#grubCutによる自動画像分割
mask = np.zeros(img.shape[:2],np.uint8)

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (0,0,400,300)  # 左領域が内部に含まれるように矩形指定　左右境界を指定している訳ではないことに注意
cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
cv2_imshow(mask2*255)
print('図11　自動生成された分割用マスク')

# 左右の画像　領域外は黒
imgL = img*mask2[:,:,np.newaxis]
imgR = img*(1-mask2[:,:,np.newaxis])

# 左右それぞれの画像のヒストグラム生成
histL = cv2.calcHist([imgL],[0],None,[256],[0,256])
# どちらも対象外領域である黒以外の最大画素数のレベル（背景色）を取り出す。
majorL = np.argmax(histL[1:])-1
histR = cv2.calcHist([imgR],[0],None,[256],[0,256])
majorR = np.argmax(histR[1:])-1

oneimg = np.ones((mask2.shape[0],mask2.shape[1],3),np.uint8) # 明るさ１の画像
baseL = majorL*oneimg*(1-mask2[:,:,np.newaxis]) # 左画像の右を左の背景色で塗る
baseR = majorR*oneimg*mask2[:,:,np.newaxis] # 右画像の左を右の背景色で塗る
imgL = cv2.add(imgL,baseL)
imgR = cv2.add(imgR,baseR)

cv2_imshow(imgL)
print('図12　左部分画像')
cv2_imshow(imgR)
print('図13　右部分画像')

In [0]:
# OTSUの方法により、左右それぞれの最適しきい値を求める
retL,binL = cv2.threshold(cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY),0,255,cv2.THRESH_OTSU)
cv2_imshow(binL)
print('図14　左画像を判別分析法で２階調化　　左のしきい値',retL)
retR,binR = cv2.threshold(cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY),0,255,cv2.THRESH_OTSU)
cv2_imshow(binR)
print('図15　右画像を判別分析法で２階調化　　右のしきい値',retR)
cv2_imshow(cv2.min(binL,binR))
print('図16　　結果の合成')

# pict03 もっと難しい画像にチャレンジ¶


In [0]:
url3 = "https://monosnap.com/image/Z2p7IYz0mfZUHfI2SJE5zVIZyHJt0I" # 画像URL
img_path, header = urllib.request.urlretrieve(url3)
img3 = cv2.imread(img_path,0) #  画像の読み込み
cv2_imshow(img3) #
print("図17  処理対象画像 pict03")
cv2.imwrite("入力画像.png",img3)
  
hist3 = cv2.calcHist([img3],[0],None,[256],[0,256])
plt.hist(img3.ravel(),256,[0,256]); plt.show()
print("図18　pict03 の　ヒストグラム")

In [0]:
#@title Number fields
offset = 14 #@param {type:"slider", min:-30, max:20, step:1}
ksize = 27 #@param {type:"slider", min:3, max:40, step:2}
th3 = cv2.adaptiveThreshold(img3,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,ksize,offset)
cv2_imshow(th3) #
print("          図19     移動平均法の結果　")

In [0]:
img3 = cv2.imread(img_path,1)
cv2_imshow(img3)
print('図20　入力画像　pict03 （再掲）')

#grubCutによる自動画像分割
mask = np.zeros(img3.shape[:2],np.uint8)

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (0,0,500,650)  # 左領域が内部に含まれるように矩形指定　左右境界を指定している訳ではないことに注意
cv2.grabCut(img3,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
cv2_imshow(mask2*255)
print('図21　自動生成された分割用マスク')

# 左右の画像　領域外は黒
imgL = img3*mask2[:,:,np.newaxis]
imgR = img3*(1-mask2[:,:,np.newaxis])

# 左右それぞれの画像のヒストグラム生成
histL = cv2.calcHist([imgL],[0],None,[256],[0,256])
# どちらも対象外領域である黒以外の最大画素数のレベル（背景色）を取り出す。
majorL = np.argmax(histL[1:])-1
histR = cv2.calcHist([imgR],[0],None,[256],[0,256])
majorR = np.argmax(histR[1:])-1

oneimg = np.ones((mask2.shape[0],mask2.shape[1],3),np.uint8) # 明るさ１の画像
baseL = majorL*oneimg*(1-mask2[:,:,np.newaxis]) # 左画像の右を左の背景色で塗る
baseR = majorR*oneimg*mask2[:,:,np.newaxis] # 右画像の左を右の背景色で塗る
imgL = cv2.add(imgL,baseL)
imgR = cv2.add(imgR,baseR)

cv2_imshow(imgL)
print('図22　左部分画像')
cv2_imshow(imgR)
print('図23　右部分画像')

In [0]:
# OTSUの方法によ理、左右それぞれの最適しきい値を求める
retL,binL = cv2.threshold(cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY),0,255,cv2.THRESH_OTSU)
cv2_imshow(binL)
print('図24　左画像を判別分析法で２階調化　　左のしきい値',retL)
retR,binR = cv2.threshold(cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY),0,255,cv2.THRESH_OTSU)
cv2_imshow(binR)
print('図25　右画像を判別分析法で２階調化　　右のしきい値',retR)
cv2_imshow(cv2.min(binL,binR))
print('図26　　結果の合成 pcit03')